# 📁 03 - Feature Selection

### 🎯 Objective
Since this time series problem can also be approached as a tabular regression task, the goal of this notebook is to identify which features contribute the most to the prediction of the target variable (`stress_score`) by evaluating their relevance and statistical significance.

---

### 🧠 Why Tabular First?
Although this is fundamentally a time series problem, treating it as a tabular problem allows us to:
- Detect useful **patterns and relationships**
- Quickly evaluate **feature importance** via models like Random Forest or XGBoost
- Test **lagged variables** and their predictive power
- Analyze **multicollinearity** (e.g., via VIF)
- Use **OLS regression** to inspect coefficients, p-values, and R²

This helps us to **narrow down the feature space** before moving into deep learning or time-series-specific models (e.g., LSTM, ARIMA).

---

### ⚙️ Key Steps

1. **Train Tree-Based Models**  
   - Use Random Forest to compute feature importances
   - Test parameter tuning with `RandomizedSearchCV`

2. **Lag Analysis**  
   - Test lagged versions (1–5 days) of the most important features  
   - Observe performance changes and feature importance evolution

3. **OLS Regression & Multicollinearity**  
   - Fit a Multiple Linear Regression model  
   - Use p-values and adjusted R² for insight  
   - Analyze VIF to detect multicollinearity

4. **Final Selection**  
   - Keep the most informative features  
   - Prepare a lightweight dataset to be passed into the modeling stage

---

### 📦 Output
- `data-for-model.csv`: A reduced dataset containing only the features that showed predictive power and low multicollinearity.

---

> 📝 Note: This notebook acts as a bridge between EDA and model training. The idea is to simplify the dataset for better model performance and interpretability.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import warnings

#Suppress scientific notation
np.set_printoptions(suppress=True)
#Supress warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../data/processed/data.csv', sep=',')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.columns

In [ ]:
# Very basic train test split based on date:
x_train = data[data["date"] < "2025-01-01"]
y_train = x_train['stress_score']
# Here we need to drop stress_score becuase it's the target and the date becuase if we treat as tabular data doesn't matter:
x_train.drop(columns=['stress_score', 'date'], inplace=True)
x_test = data[data["date"] >= "2025-01-01"]
y_test = x_test['stress_score']
# Same here:
x_test.drop(columns=['stress_score', 'date'], inplace=True)


In [ ]:
#First we can try a RF as default so:
rf = RandomForestRegressor(random_state=42)

rf.fit(x_train, y_train)
preds = rf.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f'RMSE of Random Forest Regressor (Default): {np.round(rmse, 4)}')

In [ ]:
features = pd.Series(rf.feature_importances_, index=rf.feature_names_in_)
features.sort_values(ascending=False).head(10).plot(kind='barh', figsize=(10, 8))
plt.title("Top 10 Feature Importances")
plt.show()

In [ ]:
features.sort_values(ascending=False).head(5)

In [ ]:
# Then we can try tunning Hyperparameters through Random Search CV:
rf = RandomForestRegressor()

params = {'n_estimators': [10, 50, 75, 100, 150],
          'max_depth': [2, 4, 6, 8],
          'min_samples_split': [2, 4, 6, 8],
          'min_samples_leaf': [1, 2, 3, 4],
          'max_features': [0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1], #1 means all features   
}

rf_best = RandomizedSearchCV(estimator=rf, 
                             param_distributions=params,
                             n_iter=10,
                             scoring='neg_root_mean_squared_error',
                             n_jobs=-1,
                             random_state=42)

In [ ]:
rf_best.fit(x_train,y_train)

In [ ]:
preds = rf_best.predict(x_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f'RMSE of Random Forest Regressor (Tuned):: {np.round(rmse, 4)}')

In [ ]:
# When you run RandomizedSearchCV, the resulting object rf_best is not the trained model itself — it's a wrapper around it.
rf_best = rf_best.best_estimator_

In [ ]:
features = pd.Series(rf_best.feature_importances_, index=rf_best.feature_names_in_)
features.sort_values(ascending=False).head(10).plot(kind='barh', figsize=(10, 8))
plt.title("Top 10 Feature Importances")
plt.show()

In [ ]:
features.sort_values(ascending=False).head(5)

In [ ]:
# Now we can keep the five more important features (RF with less RMSE) and lag them in order to see if we improve the performance.
# Also we need to keep the date in order to do the train test split and the target too. 

data = data[['date', 'stress_max', 'stress_min',
             'heart_rate', 'heart_min_rate',
             'heart_max_rate', 'stress_score'
             ]]

In [ ]:
data.head()

In [ ]:
data['stress_max_lag1'] = data['stress_max'].shift(1)
data['stress_max_lag2'] = data['stress_max'].shift(2)
data['stress_max_lag3'] = data['stress_max'].shift(3)
data['stress_max_lag4'] = data['stress_max'].shift(4)
data['stress_max_lag5'] = data['stress_max'].shift(5)

data['stress_min_lag1'] = data['stress_min'].shift(1)
data['stress_min_lag2'] = data['stress_min'].shift(2)
data['stress_min_lag3'] = data['stress_min'].shift(3)
data['stress_min_lag4'] = data['stress_min'].shift(4)
data['stress_min_lag5'] = data['stress_min'].shift(5)

data['heart_rate_lag1'] = data['heart_rate'].shift(1)
data['heart_rate_lag2'] = data['heart_rate'].shift(2)
data['heart_rate_lag3'] = data['heart_rate'].shift(3)
data['heart_rate_lag4'] = data['heart_rate'].shift(4)
data['heart_rate_lag5'] = data['heart_rate'].shift(5)

data['heart_min_rate_lag1'] = data['heart_min_rate'].shift(1)
data['heart_min_rate_lag2'] = data['heart_min_rate'].shift(2)
data['heart_min_rate_lag3'] = data['heart_min_rate'].shift(3)
data['heart_min_rate_lag4'] = data['heart_min_rate'].shift(4)
data['heart_min_rate_lag5'] = data['heart_min_rate'].shift(5)

data['heart_max_rate_lag1'] = data['heart_max_rate'].shift(1)
data['heart_max_rate_lag2'] = data['heart_max_rate'].shift(2)
data['heart_max_rate_lag3'] = data['heart_max_rate'].shift(3)
data['heart_max_rate_lag4'] = data['heart_max_rate'].shift(4)
data['heart_max_rate_lag5'] = data['heart_max_rate'].shift(5)

In [ ]:
# Drop rows with any NaN *after* all shifts
data.dropna(inplace=True)

In [ ]:
data.head()

In [ ]:
# Sanity check:
data.isna().sum()

In [ ]:
# Same train test split as before with this 'new' dataset:

x_train = data[data["date"] < "2025-01-01"]
y_train = x_train['stress_score']
x_train.drop(columns=['stress_score', 'date'], inplace=True)

x_test = data[data["date"] >= "2025-01-01"]
y_test = x_test['stress_score']
x_test.drop(columns=['stress_score','date'], inplace=True)

In [ ]:
# A default RF I think it's enough to compare with the preview one:
rf = RandomForestRegressor(random_state=42)

rf.fit(x_train, y_train)
preds = rf.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f'RMSE of Random Forest Regressor (Default): {np.round(rmse, 4)}')

In [ ]:
features = pd.Series(rf.feature_importances_, index=rf.feature_names_in_)
features.sort_values(ascending=False).head(10).plot(kind='barh', figsize=(10, 8))
plt.title("Top 10 Feature Importances")
plt.show()

In [ ]:
features.sort_values(ascending=False).head(10)

In [ ]:
# Statsmodel is more clearly for statistical representation than Sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
import statsmodels.api as sm

# Use same train-test split
X_train_lr = x_train.copy()
X_test_lr = x_test.copy()

# Add constant for statsmodels
X_train_sm = sm.add_constant(X_train_lr)
model = sm.OLS(y_train, X_train_sm).fit()
print(model.summary())  # this shows adjusted R2, p-values, confidence intervals

# Predict and get RMSE
X_test_sm = sm.add_constant(X_test_lr)
y_pred = model.predict(X_test_sm)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse:.4f}")


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# X: your features (without the target), and including the constant
X = sm.add_constant(x_train)

vif = pd.DataFrame()
vif['feature'] = X.columns
# Parameters exog{ndarray, DataFrame} and exog_idx int => index of the exogenous variable in the columns of exog
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif.sort_values(by='VIF', ascending=False))

In [ ]:
# We will reload the data in order to recuperate the rows missed on shifted procedure 
# and do it again only for 3 lags and at the end save it with the features selected:

data = pd.read_csv('../data/processed/data.csv', sep=',', 
                   usecols=['date', 'stress_max', 'stress_min',
             'heart_rate', 'heart_min_rate',
             'heart_max_rate', 'stress_score'])

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data['heart_min_rate_lag1'] = data['heart_min_rate'].shift(1)
data['heart_min_rate_lag2'] = data['heart_min_rate'].shift(2)
data['heart_min_rate_lag3'] = data['heart_min_rate'].shift(3)

data.dropna(inplace=True)

In [ ]:
data.to_csv('../data/processed/data-for-model.csv')

## 📌 Insights Summary

- **Stress-related features dominate prediction:**  
  `stress_max`, `stress_min`, and `heart_rate` show the strongest correlation with `stress_score`.

- **Lagged features matter:**  
  Adding lags (1–3 days) of min heart rate improved RMSE significantly in Random Forest.

- **Sleep metrics underperformed:**  
  No strong predictive power from `sleep_score`, `sleep_efficiency`, or recovery indicators, possibly due to missing data or weak relationship.

- **Multicollinearity warning:**  
  VIF analysis revealed redundancy among lagged `heart_rate` variables — need for dimensionality control.

These findings informed the choice to proceed with time series–oriented models and reduced feature sets for better generalization and interpretability.